In [40]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pyaudio
import warnings
warnings.filterwarnings('ignore')

*mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound*

*chroma: Pertains to the 12 different pitch classes*

*mel: Mel Spectrogram Frequency*

In [41]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))   #stft(short term fourier transform) calculates the magnitude of frequency
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [42]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
observed_emotions=['calm', 'happy', 'fearful', 'disgust', 'angry', 'disgust']

In [43]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("speech-emotion-recognition-ravdess-data//Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)
def displayEmotion(emotion):
    print(emotions[emotion])

In [44]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [45]:
print((x_train.shape[0], x_test.shape[0]))

(720, 240)


In [46]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [47]:
model=MLPClassifier(alpha=0.01, batch_size=128, epsilon=1e-08, hidden_layer_sizes=(200,), learning_rate='adaptive', max_iter=500)

In [48]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=128, hidden_layer_sizes=(200,),
              learning_rate='adaptive', max_iter=500)

In [49]:
y_pred=model.predict(x_test)

In [50]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.83%


In [51]:
for i in range(240):
    print('{}) Prediction = {}\n   True= {}'.format(i,y_pred[i],y_test[i]))

0) Prediction = calm
   True= angry
1) Prediction = happy
   True= happy
2) Prediction = fearful
   True= fearful
3) Prediction = disgust
   True= calm
4) Prediction = happy
   True= angry
5) Prediction = fearful
   True= fearful
6) Prediction = fearful
   True= calm
7) Prediction = disgust
   True= disgust
8) Prediction = calm
   True= calm
9) Prediction = fearful
   True= fearful
10) Prediction = fearful
   True= fearful
11) Prediction = calm
   True= calm
12) Prediction = happy
   True= disgust
13) Prediction = disgust
   True= disgust
14) Prediction = fearful
   True= angry
15) Prediction = disgust
   True= calm
16) Prediction = disgust
   True= disgust
17) Prediction = calm
   True= calm
18) Prediction = calm
   True= calm
19) Prediction = happy
   True= happy
20) Prediction = fearful
   True= disgust
21) Prediction = happy
   True= happy
22) Prediction = calm
   True= calm
23) Prediction = calm
   True= calm
24) Prediction = angry
   True= disgust
25) Prediction = calm
   True= c

In [14]:
def emotionRecognition(audioFile, mfcc=False, chroma=False, mel=False):
    emotion=audioFile.split("-")[2]
    with open(audioFile) as audioFile:
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    displayEmotion(emotion)

In [15]:
audioFile = "12-01-07-02-02-01.wav"
emotionRecognition(audioFile)

disgust
